# Performance competition

Code to detecte the neighbouring bubble, and choose the one with the biggest velocity magnitude

bnc:      bubble neighbouring cells, the neighbouring cells for the cell where the bubble is now located;

bc:       bubble cell, the cell ID which the bubble is now in;

velocity: bubble velocity, shape=(n,3), n is total bubble number

bp:       bubble partner, shape=(n,), which indicates the chosen bubble partner

count:    shape=(n,): how many bubbles stay in the neighouring cells for each bubble

In [16]:
# this is a numpy version

import numpy

def nsearch(bnc, bc, velocity):
    l, w = bnc.shape
    bp = np.zeros(l, dtype="int64") - 1
    count = np.zeros(l, dtype="int64")
    
    for main in range(l):
        for partner in range(main+1, l):
            partnerID = bc[partner]
            if partnerID in bnc[main, :]:
                if bp[main] == -1:
                    bp[main] = partner
                else:
                    tempoz = np.sum(velocity[bp[main], :]*velocity[bp[main], :])
                    if tempoz < np.sum(velocity[partner, :]*velocity[partner, :]):
                        bp[main] = partner
                count[main] += 1

    return bp, count

In [17]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [18]:
%%cython
# this is a cython view with optimized memoryview and variable type defining

import cython
import numpy as np
cimport numpy as np

cdef void csearch(long [:,:] bnc, long [:] bc, double [:,:] velocity, long [:] bp, long [:] count):
    cdef long main, partner, l, w, i
    
    l = bnc.shape[0]
    w = bnc.shape[1]
    
    for main in range(l):
        for partner in range(main+1, l):
            partnerID = bc[partner]
            for i in range(w):
                if partnerID == bnc[main, i]:
                    if bp[main] == -1:
                        bp[main] = partner
                    else:
                        tempoz = velocity[bp[main], 0]*velocity[bp[main], 0] + \
                                 velocity[bp[main], 1]*velocity[bp[main], 1] + \
                                 velocity[bp[main], 2]*velocity[bp[main], 2]
                                
                        if tempoz < velocity[partner, 0]*velocity[partner, 0] + \
                                    velocity[partner, 1]*velocity[partner, 1] + \
                                    velocity[partner, 2]*velocity[partner, 2]:
                            bp[main] = partner
                    count[main] += 1
                    break
                    
def csearch1(bnc, bc, velocity, bp, count):
    csearch(bnc, bc, velocity, bp, count)

In [19]:
%%cython
# this is a cython version with numpy provided array iterator np.nditer, which enables efficient elementwise array operation.

import cython
import numpy as np
cimport numpy as np

DTYPE = np.int
ctypedef np.int_t DTYPE_t

# np.ndarray[DTYPE_t, ndim=1]

@cython.boundscheck(False)
def csearch2(long [:,:] bnc, long [:] bc, double [:,:] velocity):
    cdef long size, w, i, j, k, partnerID
    cdef double tempoz
    cdef np.ndarray[long] rbc
    cdef np.ndarray[long] rbnc
    cdef np.ndarray[long] wbp
    cdef np.ndarray[long] wcount
    cdef np.ndarray[double] rvel
    
    w = bnc.shape[1]
    
    it = np.nditer([bc, None, None], flags=['reduce_ok', 'external_loop','buffered', 'delay_bufalloc'],
                    op_flags=[['readonly'], ['readwrite', 'allocate'], ['readwrite', 'allocate']])
    it.operands[1][...] = -1
    it.operands[2][...] = 0
    it.reset()
    
    read_bnc = np.nditer([bnc], flags=['external_loop','buffered'],op_flags=[['readonly']])
    read_vel = np.nditer([velocity], flags=['external_loop','buffered'],op_flags=[['readonly']])

    for array_bc, array_bp, array_count in it:
        rbc    = array_bc
        wbp    = array_bp
        wcount = array_count
        rbnc   = read_bnc[0]
        rvel   = read_vel[0]
        
        size   = rbc.shape[0]
        for i in range(size):
            for j in range(i+1, size):
                partnerID = rbc[j]
                for k in range(w):
                    if partnerID == rbnc[i*w+k]:
                        if wbp[i] == -1:
                            wbp[i] = j
                        else:
                            tempoz = rvel[wbp[i]*3]*rvel[wbp[i]*3] + \
                                     rvel[wbp[i]*3+1]*rvel[wbp[i]*3+1] + \
                                     rvel[wbp[i]*3+2]*rvel[wbp[i]*3+2]
                            if tempoz < rvel[j*3]*rvel[j*3] +\
                                        rvel[j*3+1]*rvel[j*3+1] + \
                                        rvel[j*3+2]*rvel[j*3+2]:
                                wbp[i] = j
                        wcount[i] += 1
                        break
                        
    return it.operands[1], it.operands[2]


In [20]:
%%cython
# this is a test, to see if more performance can be achieved with np.nditer

import cython
import numpy as np
cimport numpy as np

@cython.nonecheck(False)
@cython.boundscheck(False)
@cython.profile(False)
def csearch3(long [:,:] bnc, long [:] bc, double [:,:] velocity):
    cdef long size, w, i, j, k, partnerID
    cdef double tempoz
    cdef np.ndarray[long] rbc
    cdef np.ndarray[long] rbnc
    cdef np.ndarray[long] wbp
    cdef np.ndarray[long] wcount
    cdef np.ndarray[double] rvel
    cdef np.ndarray[double] rvel2
    
    w = bnc.shape[1]
    
    it = np.nditer([bc, None, None], flags=['external_loop','buffered', 'reduce_ok', 'delay_bufalloc'],
                    op_flags=[['readonly'], ['readwrite', 'allocate'], ['readwrite', 'allocate']],
                    op_dtypes=['long', 'long', 'long'])
    it.operands[1][...] = -1
    it.operands[2][...] = 0
    it.reset()
    
    read_bnc = np.nditer([bnc], flags=['external_loop','buffered'],
                         op_flags=[['readonly']], op_dtypes=['long'])
    
    read_vel = np.nditer([velocity, None], flags=['external_loop','buffered', 'reduce_ok', 'delay_bufalloc'],
                         op_flags=[['readonly'], ['readwrite', 'allocate']], op_dtypes=['double', 'double'],
                         op_axes=[[0,1], None])
    read_vel.operands[1][...] = 0
    read_vel.reset()
    
    
    for array_bc, array_bp, array_count in it:
        rbc    = array_bc
        wbp    = array_bp
        wcount = array_count
        rbnc   = read_bnc[0]
        
        for rvel, rvel2 in read_vel:
            #rvel   = read_vel[0]
            rvel2[...] = rvel*rvel

            size   = rbc.shape[0]
            for i in range(size):
                for j in range(i+1, size):
                    partnerID = rbc[j]
                    for k in range(w):
                        if partnerID == rbnc[i*w+k]:
                            if wbp[i] == -1:
                                wbp[i] = j
                            else:
                                tempoz = rvel2[wbp[i]*3] + rvel2[wbp[i]*3+1] + rvel2[wbp[i]*3+2]
                                if tempoz < rvel2[j*3] + rvel2[j*3+1] + rvel2[j*3+2]:
                                    wbp[i] = j
                            wcount[i] += 1
                            break
                        
    return it.operands[1], it.operands[2]

In [21]:
import numpy as np
import time

In [29]:
# test for 6000 bubbles
n = 6000

In [30]:
pbnc = np.random.randint(n/2,60000, size=(n,5)).astype("int64")
pbc = np.random.randint(n/2,60000, size=(n)).astype("int64")
pvelocity = np.random.rand(n,3).astype("float64")

In [31]:
bp = np.zeros(n, dtype="int64") - 1
count = np.zeros(n, dtype="int64")

In [32]:
t1 = time.time()
k1, k2 =nsearch(pbnc, pbc, pvelocity)
t2 = time.time()
csearch1(pbnc, pbc, pvelocity, bp, count)
t3 = time.time()
k3, k4 = csearch2(pbnc, pbc, pvelocity)
t4 = time.time()

In [33]:
np.all(k2==count)

True

In [34]:
np.all(k4==count)

True

In [35]:
print("Performance competition of bubble interaction detector for %s bubbles:"%n)
print("numpy + python loop: {0:7.4f} s\n"\
      "cython memoryview:   {1:7.4f} s\n"\
      "cython nditerator:   {2:7.4f} s\n".format(t2-t1,t3-t2,t4-t3))

Performance competition of bubble interaction detector for 6000 bubbles:
numpy + python loop: 61.4318 s
cython memoryview:    7.0924 s
cython nditerator:    0.2400 s



In [256]:
n = 1000
for _ in range(10):
    pbnc = np.random.randint(n/2,60000, size=(n,5)).astype("int64")
    pbc = np.random.randint(n/2,60000, size=(n)).astype("int64")
    pvelocity = np.random.rand(n,3).astype("float64")
    bp = np.zeros(n, dtype="int64") - 1
    count = np.zeros(n, dtype="int64")
    t1 = time.time()
    #search(pbnc, pbc, pvelocity, bp, count)
    k1, k2 = csearch2(pbnc, pbc, pvelocity)
    t2 = time.time()

    t3 = time.time()
    k3, k4 = csearch3(pbnc, pbc, pvelocity)
    t4 = time.time()
    
    print( "{0:2.4f}, {1:2.4f}".format(t2-t1, t4-t3) + ",\tresults the same = " + str(np.all(k2==k4)) )

0.0119, 0.0119,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0118, 0.0118,	results the same = True
0.0119, 0.0122,	results the same = True
